# Revised case normalization for Hirslanden Beau Site 2017

In [4]:
import pandas as pd
import sys
sys.path.insert(0, '/home/jovyan/work')

import os

from py.global_configs import *
from py.normalize import normalize

from src.service import bfs_cases_db_service as bfs_db

UndefinedValueError: BFS_CASES_DB_URL not found. Declare it as envvar or define a default value.

In [6]:
file_info = FileInfo(
    os.path.join(ROOT_DIR, 'raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx'),
    'Hirslanden Salem',
    '2017',
    ['Änderungen_SA_2017']
)

print(file_info)

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx', hospital_name_db='Hirslanden Salem', year='2017', sheets=['Änderungen_SA_2017'])


In [7]:
cols_to_rename = dict(COLUMNS_TO_RENAME)
# Replace 'admno' with 'fall nummer'
cols_to_rename.pop('admno')
cols_to_rename['fall nummer'] = CASE_ID_COL


df_revised_case_d2d = normalize(file_info, 0)

FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/work/src/revised_case_normalization/raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx'

In [ ]:
df_revised_case_d2d.head()

# match to the database


In [ ]:
# get the case_id from revised_case

revised_case_id = df_revised_case_d2d['case_id'].values
revised_case_id

In [ ]:
# match to the database
revised_case_db = bfs_db.get_bfs_cases_by_ids(revised_case_id)
revised_case_db.head()

In [ ]:
# 
print('There are {} out of {} revised cases from DtoD that are matched with the database for {} {}'.format(len(revised_case_db), len(revised_case_id), file.hospital_name_db, file.year))

In [ ]:
# if we find the match cases, then we need to check if the case_id, gender, year....are match